# Lectura y preprocesamiento de datos

In [1]:
import pandas as pd
import datetime
import numpy as np
from IPython.display import clear_output

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_excel("data.xlsb")
data["Fecha_Reporte"] = data["Fecha_Reporte"].apply(lambda x : pd.to_datetime("1899-12-30") + datetime.timedelta(days=x))
data["Fecha_Ocurrencia"] = data["Fecha_Ocurrencia"].apply(lambda x : pd.to_datetime("1899-12-30") + datetime.timedelta(days=x))
data["Fecha_Pago"] = data["Fecha_Pago"].apply(lambda x : pd.to_datetime("1899-12-30") + datetime.timedelta(days=x))

In [3]:
aux = data.groupby(["Fecha_Pago"])["Importe USD"].sum().reset_index()

In [4]:
fecha_inicial = aux["Fecha_Pago"].min()
fecha_final = aux["Fecha_Pago"].max()

rango_fechas = pd.date_range(fecha_inicial, fecha_final, freq="D")

In [5]:
#Si una fecha no tiene registro, se agrega con importe 0
aux = aux.set_index("Fecha_Pago")
aux = aux.reindex(rango_fechas, fill_value = 0)
aux = aux.reset_index(names = ["Fecha_Pago", "Importe USD"])

In [6]:
X = aux[["Fecha_Pago", "Importe USD"]]
y = aux[["Importe USD"]]

In [7]:
X["anno"] = X["Fecha_Pago"].apply(lambda x : x.year)
X["Mes"] = X["Fecha_Pago"].apply(lambda x : x.month)
X["Dia"] = X["Fecha_Pago"].apply(lambda x : x.day)
X["Dia_Semana"] = X["Fecha_Pago"].apply(lambda x : x.weekday())

In [8]:
#Ahora cada uno de estos valores los transformaremos a un encoding cíclico.
X["Dia_Semana_sin"] = X["Dia_Semana"].apply(lambda x : np.sin(2*np.pi*x/7))
X["Dia_Semana_cos"] = X["Dia_Semana"].apply(lambda x : np.cos(2*np.pi*x/7))

In [9]:
X["Mes_sin"] = X["Mes"].apply(lambda x : np.sin(2*np.pi*x/12))
X["Mes_cos"] = X["Mes"].apply(lambda x : np.cos(2*np.pi*x/12))

In [10]:
#Los días dependerán del mes, pues hay meses que tienen 30 días y otros 31, además de febrero que tiene 28 o 29.
X["Dia_sin"] = X.apply(lambda x : np.sin(2*np.pi*x["Dia"]/x["Fecha_Pago"].days_in_month), axis=1)
X["Dia_cos"] = X.apply(lambda x : np.cos(2*np.pi*x["Dia"]/x["Fecha_Pago"].days_in_month), axis=1)

In [11]:
#El año lo estandarizamos, pero comenzando desde 0.
X["anno"] = (X["anno"] - X["anno"].min()) / (X["anno"].max() - X["anno"].min())

In [12]:
columns = ["anno", "Dia_Semana_sin", "Dia_Semana_cos", "Mes_sin", "Mes_cos", "Dia_sin", "Dia_cos", "Importe USD"]

In [13]:
#Normalizamos los valores de y
y_mean = y.values.mean()
y_std = y.values.std()
y = (y - y_mean) / y_std
X["Importe USD"] = (X["Importe USD"] - y_mean) / y_std

In [14]:
#Creamos el X_train e y_train utilizando secuencias de un largo fijo
sequence_length = 30
X_train = []
y_train = []
X_test = []
y_test = []

for i in range(sequence_length, int(len(X) * 0.9)):
    X_train.append(X.iloc[i - sequence_length : i][columns].values)
    y_train.append(y.iloc[i])

for i in range(int(len(X) * 0.9), len(X)):
    X_test.append(X.iloc[i - sequence_length : i][columns].values)
    y_test.append(y.iloc[i])

X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [15]:
X_train.shape, y_train.shape

((4560, 30, 8), (4560, 1))

# Implementación del modelo

In [16]:
n_steps = 30
n_features = 8

In [ ]:
def create_model(n_lstm, sequence_length = 30):

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(n_lstm, input_shape=(n_steps, n_features)))
    model.add(tf.keras.layers.Dense(1))

    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
        loss = tf.keras.losses.MeanSquaredError(),
    )

    return model

In [ ]:
best_model = None
best_loss = float("inf")
loss = float("inf")

for i in range(20):

    print(f"BEST LOSS: {best_loss}")
    print(f"Iteration {i} - Loss: {loss}")

    model = create_model(64)
    model.fit(
        X_train,
        y_train,
        validation_data = (X_test, y_test),
        epochs = 1000,
        batch_size = 32,
        callbacks = [tf.keras.callbacks.EarlyStopping(patience = 50, restore_best_weights = True)])

    loss = model.evaluate(X_test, y_test)

    if loss < best_loss:
        best_model = model
        best_loss = loss

    clear_output(wait=True)

BEST LOSS: 0.20694094896316528
Iteration 19 - Loss: 0.212728813290596
Epoch 1/1000
143/143 [==============================] - 3s 11ms/step - loss: 1.0968 - val_loss: 0.2173
Epoch 2/1000
143/143 [==============================] - 1s 7ms/step - loss: 1.0949 - val_loss: 0.2159
Epoch 3/1000
143/143 [==============================] - 1s 6ms/step - loss: 1.0907 - val_loss: 0.2140
Epoch 4/1000
143/143 [==============================] - 1s 6ms/step - loss: 1.0889 - val_loss: 0.2166
Epoch 5/1000
143/143 [==============================] - 1s 7ms/step - loss: 1.0897 - val_loss: 0.2217
Epoch 6/1000
143/143 [==============================] - 1s 6ms/step - loss: 1.0859 - val_loss: 0.2172
Epoch 7/1000
143/143 [==============================] - 1s 6ms/step - loss: 1.0857 - val_loss: 0.2207
Epoch 8/1000
143/143 [==============================] - 1s 6ms/step - loss: 1.0833 - val_loss: 0.2271
Epoch 9/1000
143/143 [==============================] - 1s 6ms/step - loss: 1.0826 - val_loss: 0.2125
Epoch 10/10

In [ ]:
best_model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_17 (LSTM)              (None, 64)                18688     
                                                                 
 dense_17 (Dense)            (None, 1)                 65        
                                                                 
Total params: 18,753
Trainable params: 18,753
Non-trainable params: 0
_________________________________________________________________


In [22]:
best_model.save(f"./Models/lstm_64_{best_loss}.h5")

In [23]:
def create_model(n_lstm_1, n_lstm_2, sequence_length = 30):

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(n_lstm_1, input_shape=(n_steps, n_features), return_sequences=True))
    model.add(tf.keras.layers.LSTM(n_lstm_2))
    model.add(tf.keras.layers.Dense(1))

    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
        loss = tf.keras.losses.MeanSquaredError(),
    )

    return model

In [24]:
best_model = None
best_loss = float("inf")
loss = float("inf")

for i in range(20):

    print(f"BEST LOSS: {best_loss}")
    print(f"Iteration {i} - Loss: {loss}")

    model = create_model(64, 32)
    model.fit(
        X_train,
        y_train,
        validation_data = (X_test, y_test),
        epochs = 1000,
        batch_size = 32,
        callbacks = [tf.keras.callbacks.EarlyStopping(patience = 50, restore_best_weights = True)])

    loss = model.evaluate(X_test, y_test)

    if loss < best_loss:
        best_model = model
        best_loss = loss

    clear_output(wait=True)

BEST LOSS: 0.20665493607521057
Iteration 19 - Loss: 0.21121704578399658
Epoch 1/1000
143/143 [==============================] - 6s 18ms/step - loss: 1.0952 - val_loss: 0.2245
Epoch 2/1000
143/143 [==============================] - 1s 10ms/step - loss: 1.0911 - val_loss: 0.2191
Epoch 3/1000
143/143 [==============================] - 1s 10ms/step - loss: 1.0882 - val_loss: 0.2216
Epoch 4/1000
143/143 [==============================] - 1s 10ms/step - loss: 1.0889 - val_loss: 0.2124
Epoch 5/1000
143/143 [==============================] - 1s 10ms/step - loss: 1.0882 - val_loss: 0.2210
Epoch 6/1000
143/143 [==============================] - 1s 10ms/step - loss: 1.0870 - val_loss: 0.2258
Epoch 7/1000
143/143 [==============================] - 1s 10ms/step - loss: 1.0880 - val_loss: 0.2162
Epoch 8/1000
143/143 [==============================] - 1s 10ms/step - loss: 1.0846 - val_loss: 0.2213
Epoch 9/1000
143/143 [==============================] - 1s 10ms/step - loss: 1.0848 - val_loss: 0.2229
E

In [25]:
best_model.summary()

Model: "sequential_41"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_55 (LSTM)              (None, 30, 64)            18688     
                                                                 
 lstm_56 (LSTM)              (None, 32)                12416     
                                                                 
 dense_41 (Dense)            (None, 1)                 33        
                                                                 
Total params: 31,137
Trainable params: 31,137
Non-trainable params: 0
_________________________________________________________________


In [26]:
best_model.save(f"./Models/lstm_64_32_{best_loss}.h5")

In [17]:
def create_model(n_lstm_1, n_lstm_2, sequence_length = 30):

    inputs = tf.keras.layers.Input(shape=(n_steps, n_features))

    x = tf.keras.layers.LSTM(n_lstm_1, return_sequences=True)(inputs)
    x = tf.keras.layers.LSTM(n_lstm_2)(x)
    x = tf.keras.layers.Dense(64)(x)
    outputs = tf.keras.layers.Dense(1)(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
        loss = tf.keras.losses.MeanSquaredError(),
    )

    return model

In [21]:
best_model = None
best_loss = float("inf")
loss = float("inf")

for i in range(20):

    print(f"BEST LOSS: {best_loss}")
    print(f"Iteration {i} - Loss: {loss}")

    model = create_model(64, 32)
    model.fit(
        X_train,
        y_train,
        validation_data = (X_test, y_test),
        epochs = 1000,
        batch_size = 32,
        callbacks = [tf.keras.callbacks.EarlyStopping(patience = 50, restore_best_weights = True)])

    loss = model.evaluate(X_test, y_test)

    if loss < best_loss:
        best_model = model
        best_loss = loss

    clear_output(wait=True)

BEST LOSS: 0.20673413574695587
Iteration 19 - Loss: 0.2098938673734665
Epoch 1/1000
143/143 [==============================] - 6s 17ms/step - loss: 1.0964 - val_loss: 0.2138
Epoch 2/1000
143/143 [==============================] - 1s 10ms/step - loss: 1.0923 - val_loss: 0.2314
Epoch 3/1000
143/143 [==============================] - 1s 10ms/step - loss: 1.0915 - val_loss: 0.2255
Epoch 4/1000
143/143 [==============================] - 1s 10ms/step - loss: 1.0887 - val_loss: 0.2152
Epoch 5/1000
143/143 [==============================] - 1s 10ms/step - loss: 1.0891 - val_loss: 0.2164
Epoch 6/1000
143/143 [==============================] - 1s 10ms/step - loss: 1.0884 - val_loss: 0.2194
Epoch 7/1000
143/143 [==============================] - 1s 10ms/step - loss: 1.0872 - val_loss: 0.2223
Epoch 8/1000
143/143 [==============================] - 1s 10ms/step - loss: 1.0868 - val_loss: 0.2314
Epoch 9/1000
143/143 [==============================] - 1s 10ms/step - loss: 1.0841 - val_loss: 0.2184
Ep

In [22]:
best_model.summary()

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 30, 8)]           0         
                                                                 
 lstm_32 (LSTM)              (None, 30, 64)            18688     
                                                                 
 lstm_33 (LSTM)              (None, 32)                12416     
                                                                 
 dense_32 (Dense)            (None, 64)                2112      
                                                                 
 dense_33 (Dense)            (None, 1)                 65        
                                                                 
Total params: 33,281
Trainable params: 33,281
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.save(f"./Models/lstm_64_32_64_{best_loss}.h5")

In [24]:
def create_model():

    inputs = tf.keras.layers.Input(shape=(n_steps, n_features))

    #This is going to be a RNN simple cell

    x = tf.keras.layers.SimpleRNN(64, return_sequences=True)(inputs)
    x = tf.keras.layers.SimpleRNN(32)(x)
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    outputs = tf.keras.layers.Dense(1)(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
        loss = tf.keras.losses.MeanSquaredError(),
    )

    return model

In [26]:
best_model = None
best_loss = float("inf")
loss = float("inf")

for i in range(20):

    print(f"BEST LOSS: {best_loss}")
    print(f"Iteration {i} - Loss: {loss}")

    model = create_model()
    model.fit(
        X_train,
        y_train,
        validation_data = (X_test, y_test),
        epochs = 1000,
        batch_size = 32,
        callbacks = [tf.keras.callbacks.EarlyStopping(patience = 50, restore_best_weights = True)])

    loss = model.evaluate(X_test, y_test)

    if loss < best_loss:
        best_model = model
        best_loss = loss

    clear_output(wait=True)

BEST LOSS: 0.1221190094947815
Iteration 19 - Loss: 0.20685075223445892
Epoch 1/1000
143/143 [==============================] - 12s 71ms/step - loss: 1.1256 - val_loss: 0.2046
Epoch 2/1000
143/143 [==============================] - 10s 67ms/step - loss: 1.1062 - val_loss: 0.2483
Epoch 3/1000
143/143 [==============================] - 10s 67ms/step - loss: 1.1048 - val_loss: 0.2195
Epoch 4/1000
143/143 [==============================] - 10s 68ms/step - loss: 1.1004 - val_loss: 0.2227
Epoch 5/1000
143/143 [==============================] - 10s 68ms/step - loss: 1.0888 - val_loss: 0.2141
Epoch 6/1000
143/143 [==============================] - 10s 67ms/step - loss: 1.0912 - val_loss: 0.2501
Epoch 7/1000
143/143 [==============================] - 10s 68ms/step - loss: 1.0839 - val_loss: 0.3217
Epoch 8/1000
143/143 [==============================] - 10s 67ms/step - loss: 1.1068 - val_loss: 0.2206
Epoch 9/1000
143/143 [==============================] - 10s 69ms/step - loss: 1.0817 - val_loss: 

In [27]:
best_model.summary()

Model: "model_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_26 (InputLayer)       [(None, 30, 8)]           0         
                                                                 
 simple_rnn_8 (SimpleRNN)    (None, 30, 64)            4672      
                                                                 
 simple_rnn_9 (SimpleRNN)    (None, 32)                3104      
                                                                 
 dense_50 (Dense)            (None, 64)                2112      
                                                                 
 dense_51 (Dense)            (None, 1)                 65        
                                                                 
Total params: 9,953
Trainable params: 9,953
Non-trainable params: 0
_________________________________________________________________


In [28]:
best_model.save(f"./Models/rnn_64_32_64_{best_loss}.h5")

In [29]:
def create_model():

    inputs = tf.keras.layers.Input(shape=(n_steps, n_features))

    x = tf.keras.layers.SimpleRNN(64, return_sequences=True)(inputs)
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    outputs = tf.keras.layers.Dense(1)(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
        loss = tf.keras.losses.MeanSquaredError(),
    )

    return model

In [30]:
best_model = None
best_loss = float("inf")
loss = float("inf")

for i in range(20):

    print(f"BEST LOSS: {best_loss}")
    print(f"Iteration {i} - Loss: {loss}")

    model = create_model()
    model.fit(
        X_train,
        y_train,
        validation_data = (X_test, y_test),
        epochs = 1000,
        batch_size = 32,
        callbacks = [tf.keras.callbacks.EarlyStopping(patience = 50, restore_best_weights = True)])

    loss = model.evaluate(X_test, y_test)

    if loss < best_loss:
        best_model = model
        best_loss = loss

    clear_output(wait=True)

BEST LOSS: 0.21163050830364227
Iteration 19 - Loss: 0.21296530961990356
Epoch 1/1000
143/143 [==============================] - 5s 31ms/step - loss: 1.1065 - val_loss: 0.2843
Epoch 2/1000
143/143 [==============================] - 4s 29ms/step - loss: 1.1033 - val_loss: 0.2124
Epoch 3/1000
143/143 [==============================] - 4s 29ms/step - loss: 1.0938 - val_loss: 0.2404
Epoch 4/1000
143/143 [==============================] - 4s 29ms/step - loss: 1.0944 - val_loss: 0.2691
Epoch 5/1000
143/143 [==============================] - 4s 30ms/step - loss: 1.0964 - val_loss: 0.2417
Epoch 6/1000
143/143 [==============================] - 4s 29ms/step - loss: 1.0928 - val_loss: 0.2296
Epoch 7/1000
143/143 [==============================] - 4s 29ms/step - loss: 1.0921 - val_loss: 0.2154
Epoch 8/1000
143/143 [==============================] - 4s 30ms/step - loss: 1.0939 - val_loss: 0.2365
Epoch 9/1000
143/143 [==============================] - 4s 30ms/step - loss: 1.0931 - val_loss: 0.2134
E

In [31]:
best_model.summary()

Model: "model_51"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_52 (InputLayer)       [(None, 30, 8)]           0         
                                                                 
 simple_rnn_50 (SimpleRNN)   (None, 30, 64)            4672      
                                                                 
 dense_102 (Dense)           (None, 30, 64)            4160      
                                                                 
 dense_103 (Dense)           (None, 30, 1)             65        
                                                                 
Total params: 8,897
Trainable params: 8,897
Non-trainable params: 0
_________________________________________________________________


In [32]:
best_model.save(f"./Models/rnn_64_64_{best_loss}.h5")

In [ ]:
def create_model():

    inputs = tf.keras.layers.Input(shape=(n_steps, n_features))

    x = tf.keras.layers.SimpleRNN(128, return_sequences=True)(inputs)
    x = tf.keras.layers.SimpleRNN(64)(x)
    x = tf.keras.layers.Dense(128, activation="relu")(x)
    outputs = tf.keras.layers.Dense(1)(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
        loss = tf.keras.losses.MeanSquaredError(),
    )

    return model

In [33]:
best_model = None
best_loss = float("inf")
loss = float("inf")

for i in range(20):

    print(f"BEST LOSS: {best_loss}")
    print(f"Iteration {i} - Loss: {loss}")

    model = create_model()
    model.fit(
        X_train,
        y_train,
        validation_data = (X_test, y_test),
        epochs = 1000,
        batch_size = 32,
        callbacks = [tf.keras.callbacks.EarlyStopping(patience = 50, restore_best_weights = True)])

    loss = model.evaluate(X_test, y_test)

    if loss < best_loss:
        best_model = model
        best_loss = loss

    clear_output(wait=True)

BEST LOSS: 0.21064460277557373
Iteration 19 - Loss: 0.21632631123065948
Epoch 1/1000
143/143 [==============================] - 6s 31ms/step - loss: 1.1083 - val_loss: 0.2178
Epoch 2/1000
143/143 [==============================] - 4s 29ms/step - loss: 1.0995 - val_loss: 0.2687
Epoch 3/1000
143/143 [==============================] - 4s 30ms/step - loss: 1.0963 - val_loss: 0.2159
Epoch 4/1000
143/143 [==============================] - 4s 30ms/step - loss: 1.0940 - val_loss: 0.2426
Epoch 5/1000
143/143 [==============================] - 4s 30ms/step - loss: 1.0931 - val_loss: 0.2673
Epoch 6/1000
143/143 [==============================] - 4s 30ms/step - loss: 1.0910 - val_loss: 0.2274
Epoch 7/1000
143/143 [==============================] - 4s 29ms/step - loss: 1.0916 - val_loss: 0.2436
Epoch 8/1000
143/143 [==============================] - 4s 30ms/step - loss: 1.0903 - val_loss: 0.2165
Epoch 9/1000
143/143 [==============================] - 4s 30ms/step - loss: 1.0897 - val_loss: 0.2187
E

In [34]:
best_model.summary()

Model: "model_61"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_62 (InputLayer)       [(None, 30, 8)]           0         
                                                                 
 simple_rnn_60 (SimpleRNN)   (None, 30, 64)            4672      
                                                                 
 dense_122 (Dense)           (None, 30, 64)            4160      
                                                                 
 dense_123 (Dense)           (None, 30, 1)             65        
                                                                 
Total params: 8,897
Trainable params: 8,897
Non-trainable params: 0
_________________________________________________________________


In [35]:
best_model.save(f"./Models/rnn_128_64_128_{best_loss}.h5")

In [ ]:
def create_model():

    inputs = tf.keras.layers.Input(shape=(n_steps, n_features))

    #This is going to be a RNN simple cell

    x = tf.keras.layers.SimpleRNN(64, return_sequences=True)(inputs)
    x = tf.keras.layers.SimpleRNN(32)(x)
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    outputs = tf.keras.layers.Dense(1)(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
        loss = tf.keras.losses.MeanSquaredError(),
    )

    return model